파이토치에서는 데이터를 좀 더 쉽게 다룰 수 있도록 유용한 도구로서 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공합니다. 이를 사용하면 미니 배치 학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있습니다. 기본적인 사용 방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것입니다.

Dataset을 커스텀하여 만들 수도 있지만 여기서는 텐서를 입력받아 Dataset의 형태로 변환해주는 TensorDataset을 사용해보겠습니다.

실습을 위해 기본적으로 필요한 파이토치의 도구들을 임포트합니다.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import torch.nn.functional as F

TensorDataset은 기본적으로 텐서를 입력으로 받습니다. 텐서 형태로 데이터를 정의합니다.



In [2]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

이제 이를 TensorDataset의 입력으로 사용하고 dataset으로 저장합니다.

In [3]:
dataset = TensorDataset(x_train, y_train)

파이토치의 데이터셋을 만들었다면 데이터로더를 사용 가능합니다. 데이터로더는 기본적으로 2개의 인자를 입력받는다. 하나는 데이터셋, 미니 배치의 크기입니다. 이때 미니 배치의 크기는 통상적으로 2의 배수를 사용합니다. (ex) 64, 128, 256...) 그리고 추가적으로 많이 사용되는 인자로 shuffle이 있습니다. shuffle=True를 선택하면 Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿉니다.

사람도 같은 문제지를 계속 풀면 어느 순간 문제의 순서에 익숙해질 수 있습니다. 예를 들어 어떤 문제지의 12번 문제를 풀면서, '13번 문제가 뭔지는 기억은 안 나지만 어제 풀었던 기억으로 정답은 5번이었던 것 같은데' 하면서 문제 자체보단 순서에 익숙해질 수 있다는 것입니다. 그럴 때 문제지를 풀 때마다 문제 순서를 랜덤으로 바꾸면 도움이 될 겁니다. 마찬가지로 모델이 데이터셋의 순서에 익숙해지는 것을 방지하여 학습할 때는 이 옵션을 True를 주는 것을 권장합니다.

In [5]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

이제 모델과 옵티마이저를 설계합니다. nn.Linear(input_dim, output_dim)이므로 입력층에 뉴런이 3개, 출력층에 뉴런이 1개인 구조입니다. 그리고 하나의 뉴런에는 하나의 숫자가 맵핑되므로 선형 회귀 식으로 나열한다면 다음과 같은 식이 나올겁니다.  

$H(x) = w_{1}x_{1} + w_{2}x_{2} + w_{3}x_{3} + b$

In [6]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

학습을 할 때는 각 에포크마다 데이터로더로부터 배치 크기만큼(이 예제에서는 2)의 데이터를 꺼내서 학습을 진행합니다. 배치 데이터에 대한 이해를 위해서 배치 크기만큼의 데이터를 꺼내서 사용할 때마다 크기를 출력해보았습니다. 그리고 각 배치 데이터마다의 손실을 계속 출력하고, 에포크가 끝났을 때는 해당 배치 데이터마다의 손실을 평균내어서 해당 에포크의 손실로 판단합니다.

In [ ]:
nb_epochs = 2
for epoch in range(1, nb_epochs + 1):
  train_loss = 0
  print('-' * 5, epoch, 'Epoch 진행', '-' * 45)
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    print(batch_idx + 1,'배치 데이터의 크기 :', x_train.shape)

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    train_loss += cost.item()

    print('해당 배치의 Loss: {:.6f}'.format(cost.item()))

  train_loss /= len(dataloader)
  print()
  print(f'==> Epoch Loss: {train_loss:.4f}')

  # 해당 배치 기준으로 loss 를 구함 

----- 1 Epoch 진행 ---------------------------------------------
1 배치 데이터의 크기 : torch.Size([2, 3])
해당 배치의 Loss: 51028.203125
2 배치 데이터의 크기 : torch.Size([2, 3])
해당 배치의 Loss: 22942.742188
3 배치 데이터의 크기 : torch.Size([1, 3])
해당 배치의 Loss: 2832.570801

==> Epoch Loss: 25601.1720
----- 2 Epoch 진행 ---------------------------------------------
1 배치 데이터의 크기 : torch.Size([2, 3])
해당 배치의 Loss: 2230.581055
2 배치 데이터의 크기 : torch.Size([2, 3])
해당 배치의 Loss: 471.627136
3 배치 데이터의 크기 : torch.Size([1, 3])
해당 배치의 Loss: 90.389214

==> Epoch Loss: 930.8658
